# Navier Stokes Equatoin

In [1]:
import numpy as np
import time
import scipy.sparse.linalg as sp_la
import matplotlib.pyplot as plt

# --- Core imports ---
from pycutfem.core.mesh import Mesh
from pycutfem.core.dofhandler import DofHandler
from pycutfem.utils.meshgen import structured_quad

# --- UFL-like imports ---
from pycutfem.ufl.functionspace import FunctionSpace
from pycutfem.ufl.expressions import (
    TrialFunction, TestFunction, VectorTrialFunction, VectorTestFunction,
    Function, VectorFunction, Constant, grad, inner, dot, div
)
from pycutfem.ufl.measures import dx
from pycutfem.ufl.forms import BoundaryCondition, assemble_form
from pycutfem.fem.mixedelement import MixedElement

# ============================================================================
#    NEW: Verification Data and Plotting Function
# ============================================================================

# Digitized data from Ghia, Ghia, and Shin (1982), Table 1 for Re=100
ghia_data_re100 = {
    'y_locations': np.array([0.0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0]),
    'u_velocity_on_vertical_centerline': np.array([0.0, -0.0722, -0.1364, -0.2282, -0.2928, -0.3239, -0.3273, -0.3017, -0.2452, -0.1553, -0.0524, 0.0033, 1.0]),
    'x_locations': np.array([0.0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0]),
    'v_velocity_on_horizontal_centerline': np.array([0.0, 0.0886, 0.1608, 0.2804, 0.3556, 0.3789, 0.3547, 0.2971, 0.2223, 0.1463, 0.0712, 0.0396, 0.0])
}

def create_verification_plot(dof_handler, u_solution, reference_data):
    """
    Extracts velocity profiles from the solution and plots them against
    Ghia et al. reference data for verification.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle("Verification Against Ghia, Ghia & Shin (1982) for Re=100", fontsize=16)

    # --- 1. u-velocity on vertical centerline (x=0.5) ---
    ux_dof_coords = dof_handler.get_dof_coords('ux')
    centerline_mask = np.isclose(ux_dof_coords[:, 0], 0.5)
    y_coords = ux_dof_coords[centerline_mask, 1]

    # Get all 'ux' values and filter them
    ux_values = u_solution[0].nodal_values
    u_centerline = ux_values[centerline_mask]

    # Sort for plotting
    sort_indices = np.argsort(y_coords)
    y_coords_sorted = y_coords[sort_indices]
    u_centerline_sorted = u_centerline[sort_indices]

    ax = axes[0]
    ax.plot(u_centerline_sorted, y_coords_sorted, 'b-', label='FEM Solution', lw=2)
    ax.plot(reference_data['u_velocity_on_vertical_centerline'], reference_data['y_locations'], 'ro', label='Ghia et al. (1982)', mfc='none')
    ax.set_title('u-velocity along Vertical Centerline (x=0.5)')
    ax.set_xlabel('u-velocity')
    ax.set_ylabel('Y-coordinate')
    ax.grid(True, linestyle=':')
    ax.legend()

    # --- 2. v-velocity on horizontal centerline (y=0.5) ---
    uy_dof_coords = dof_handler.get_dof_coords('uy')
    centerline_mask = np.isclose(uy_dof_coords[:, 1], 0.5)
    x_coords = uy_dof_coords[centerline_mask, 0]

    # Get all 'uy' values and filter them
    uy_values = u_solution[1].nodal_values
    v_centerline = uy_values[centerline_mask]

    # Sort for plotting
    sort_indices = np.argsort(x_coords)
    x_coords_sorted = x_coords[sort_indices]
    v_centerline_sorted = v_centerline[sort_indices]

    ax = axes[1]
    ax.plot(x_coords_sorted, v_centerline_sorted, 'b-', label='FEM Solution', lw=2)
    ax.plot(reference_data['x_locations'], reference_data['v_velocity_on_horizontal_centerline'], 'ro', label='Ghia et al. (1982)', mfc='none')
    ax.set_title('v-velocity along Horizontal Centerline (y=0.5)')
    ax.set_xlabel('X-coordinate')
    ax.set_ylabel('v-velocity')
    ax.grid(True, linestyle=':')
    ax.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()








In [2]:
# 1. ============================================================================
#    SETUP (Meshes, DofHandler, BCs)
# ===============================================================================
L, H = 1.0, 1.0
# MODIFIED: Increased resolution for better accuracy
NX, NY = 32, 32
nodes_q2, elems_q2, _, corners_q2 = structured_quad(L, H, nx=NX, ny=NY, poly_order=2)
mesh_q2 = Mesh(nodes=nodes_q2, element_connectivity=elems_q2, elements_corner_nodes=corners_q2, element_type="quad", poly_order=2)
mixed_element = MixedElement(mesh_q2, field_specs={'ux': 2, 'uy': 2, 'p': 1})

dof_handler = DofHandler(mixed_element, method='cg')

# Tag boundaries for applying BCs
bc_tags = {
    'bottom_wall': lambda x,y: np.isclose(y,0),
    'left_wall':   lambda x,y: np.isclose(x,0),
    'right_wall':  lambda x,y: np.isclose(x,L),
    'top_lid':     lambda x,y: np.isclose(y,H)
}
mesh_q2.tag_boundary_edges(bc_tags)

# Tag a single node for pressure pinning
dof_handler.tag_dof_by_locator(
    tag='pressure_pin_point', field='p',
    locator=lambda x, y: np.isclose(x, 0.0) and np.isclose(y, 0.0),
    find_first=True
)

# MODIFIED: Boundary conditions for the classic lid-driven cavity problem
bcs = [
    # No-slip on bottom, left, and right walls
    BoundaryCondition('ux', 'dirichlet', 'bottom_wall', lambda x,y: 0.0),
    BoundaryCondition('uy', 'dirichlet', 'bottom_wall', lambda x,y: 0.0),
    BoundaryCondition('ux', 'dirichlet', 'left_wall',   lambda x,y: 0.0),
    BoundaryCondition('uy', 'dirichlet', 'left_wall',   lambda x,y: 0.0),
    BoundaryCondition('ux', 'dirichlet', 'right_wall',  lambda x,y: 0.0),
    BoundaryCondition('uy', 'dirichlet', 'right_wall',  lambda x,y: 0.0),
    # Moving lid on the top wall (ux=1, uy=0)
    BoundaryCondition('ux', 'dirichlet', 'top_lid',     lambda x,y: 1.0),
    BoundaryCondition('uy', 'dirichlet', 'top_lid',     lambda x,y: 0.0),
    # Pin pressure at one point to ensure a unique solution
    BoundaryCondition('p', 'dirichlet', 'pressure_pin_point',lambda x,y: 0.0)
]

bcs_homog = [BoundaryCondition(bc.field, bc.method, bc.domain_tag, lambda x,y: 0.0) for bc in bcs]
print(f"DofHandler info: {dof_handler.info()}")

=== DofHandler (CG) ===
        ux: 4225 DOFs @ offset 0
        uy: 4225 DOFs @ offset 9
         p: 1089 DOFs @ offset 18
  total : 9539
DofHandler info: None


In [3]:
# 2. ============================================================================
#    UFL FORMULATION
# ===============================================================================
rho = Constant(1.0)
dt = Constant(0.1)
theta = Constant(1.0) # Use Backward Euler for stability (theta=1.0)
# MODIFIED: Set viscosity for Re=100
mu = Constant(0.01) # Re = (1.0 * 1.0 * 1.0) / 0.01 = 100

velocity_space = FunctionSpace("velocity", ['ux', 'uy'])
pressure_space = FunctionSpace("pressure", ['p'])

du = VectorTrialFunction(velocity_space, dof_handler=dof_handler)
dp = TrialFunction(pressure_space, dof_handler=dof_handler)
v = VectorTestFunction(velocity_space, dof_handler=dof_handler)
q = TestFunction(pressure_space, dof_handler=dof_handler)

u_k = VectorFunction(name="u_k", field_names=['ux', 'uy'], dof_handler=dof_handler)
p_k = Function(name="p_k", field_name='p', dof_handler=dof_handler)
u_n = VectorFunction(name="u_n", field_names=['ux', 'uy'], dof_handler=dof_handler)
# p_n is not needed for the steady state formulation if theta=1.0, but we keep it for consistency
p_n = Function(name="p_n", field_name='p', dof_handler=dof_handler)

# 3. ============================================================================
#    SOLVER LOOP (Modified for Steady-State Convergence)
# ===============================================================================
# --- Simulation Parameters ---
# MODIFIED: We run until steady state, not for a fixed time T_end
steady_state_tol = 1e-5
max_timesteps = 200
newton_tol = 1e-6
max_newton_iter = 15

In [ ]:
# --- Initialize Solution Functions ---
u_k.nodal_values.fill(0.0)
p_k.nodal_values.fill(0.0)
u_n.nodal_values.fill(0.0)
p_n.nodal_values.fill(0.0)

# Apply initial boundary conditions to u_n (the t=0 state)
dof_handler.apply_bcs(bcs, u_n, p_n)

# --- Main Time-Stepping Loop (to reach steady state) ---
for n in range(max_timesteps):
    t = (n + 1) * dt.value
    print(f"\n--- Solving Time Step {n+1} | t = {t:.2f}s ---")

    # Initial guess for Newton is the solution from the previous step
    u_k.nodal_values[:] = u_n.nodal_values[:]
    p_k.nodal_values[:] = p_n.nodal_values[:]
    
    # Apply non-homogeneous BCs to the current iteration guess
    dof_handler.apply_bcs(bcs, u_k, p_k)

    # --- Inner Newton Iteration Loop ---
    for k in range(max_newton_iter):
        # Jacobian and Residual definitions (using theta-method)
        jacobian = (
            rho * dot(du, v) / dt
            + theta * rho * dot(dot(grad(u_k), du), v)
            + theta * rho * dot(dot(grad(du), u_k), v)
            + theta * mu * inner(grad(du), grad(v))
            - dp * div(v)
            + q * div(du)
        ) * dx()

        residual = (
            rho * dot(u_k - u_n, v) / dt
            + theta * dot(rho * dot(grad(u_k), u_k), v)
            + (1.0 - theta) * dot(rho * dot(grad(u_n), u_n), v)
            + theta * mu * inner(grad(u_k), grad(v))
            + (1.0 - theta) * mu * inner(grad(u_n), grad(v))
            - p_k * div(v)
            + q * div(u_k)
        ) * dx()

        # We solve J*delta_U = -R
        A, R_vec = assemble_form(jacobian == -residual, dof_handler=dof_handler, bcs=bcs_homog, quad_order=6)
        
        norm_res = np.linalg.norm(R_vec)
        print(f"  Newton iteration {k+1} | Residual Norm: {norm_res:.3e}")

        if norm_res < newton_tol:
            print(f"    Newton converged in {k+1} iterations.")
            break
        
        delta_U = sp_la.spsolve(A, R_vec)
        
        dof_handler.add_to_functions(delta_U, [u_k, p_k])
        dof_handler.apply_bcs(bcs, u_k, p_k)
    else:
        raise RuntimeError(f"Newton's method did not converge after {max_newton_iter} iterations.")

    # --- Check for steady-state convergence ---
    solution_change = np.linalg.norm(u_k.nodal_values - u_n.nodal_values)
    print(f"  Change in solution (L2 norm): {solution_change:.3e}")
    if solution_change < steady_state_tol and n > 0:
        print(f"\n--- Steady state reached at t={t:.2f}s ---")
        u_n.nodal_values[:] = u_k.nodal_values[:]
        p_n.nodal_values[:] = p_k.nodal_values[:]
        break

    # Update solution for the next time step
    u_n.nodal_values[:] = u_k.nodal_values[:]
    p_n.nodal_values[:] = p_k.nodal_values[:]
else:
    print(f"\n--- Max timesteps ({max_timesteps}) reached. Solution may not be fully steady. ---")


# 4. ============================================================================
#    POST-PROCESSING AND VERIFICATION
# ===============================================================================
print("\nSimulation finished. Generating plots...")

# --- Plot Final Solution Contours ---
# The plot method from your VectorFunction class is used here.
u_n.plot(field='ux', title='U-Velocity (ux) at Steady State (Re=100)', levels=20, cmap='viridis')
u_n.plot(field='uy', title='V-Velocity (uy) at Steady State (Re=100)', levels=20, cmap='viridis')
p_n.plot(title='Pressure (p) at Steady State (Re=100)', levels=20, cmap='viridis')

# --- Generate Quantitative Verification Plot ---
create_verification_plot(dof_handler, u_n, ghia_data_re100)


--- Solving Time Step 1 | t = 0.10s ---
  Newton iteration 1 | Residual Norm: 1.130e-01
  Newton iteration 2 | Residual Norm: 2.836e-03
  Newton iteration 3 | Residual Norm: 1.653e-05
  Newton iteration 4 | Residual Norm: 2.620e-10
    Newton converged in 4 iterations.
  Change in solution (L2 norm): 5.696e+00

--- Solving Time Step 2 | t = 0.20s ---
  Newton iteration 1 | Residual Norm: 1.591e-02
  Newton iteration 2 | Residual Norm: 3.168e-04
  Newton iteration 3 | Residual Norm: 2.737e-07
    Newton converged in 3 iterations.
  Change in solution (L2 norm): 2.424e+00

--- Solving Time Step 3 | t = 0.30s ---
  Newton iteration 1 | Residual Norm: 6.531e-03
  Newton iteration 2 | Residual Norm: 1.136e-04
  Newton iteration 3 | Residual Norm: 3.427e-08
    Newton converged in 3 iterations.
  Change in solution (L2 norm): 1.533e+00

--- Solving Time Step 4 | t = 0.40s ---
  Newton iteration 1 | Residual Norm: 4.138e-03
  Newton iteration 2 | Residual Norm: 5.959e-05
  Newton iteration 3